In [3]:
import math

import torchvision
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path
from PIL import Image
from fastai.vision.all import *
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import sys
sys.path.append('../input/timm-pkg')
import timm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
ROOT = Path('../input/petfinder-pawpularity-score')
TRAIN_IMG_PATH = ROOT / 'train'

ID_COL = 'Id'
TARGET_COL = 'Pawpularity'

SEED = 42
IMG_SIZE = 512
BATCH_SIZE = 128
# N_EPOCHS = 3
# N_SPLITS = 5
# Tensorflow EfficientNet B0 Noisy-Student
MODEL_NAME = 'tf_efficientnet_b0_ns'

In [5]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(SEED)

In [6]:
train_df = pd.read_csv(ROOT / 'train.csv')
target = train_df[TARGET_COL]
train_df.head()

In [7]:
train_transform = A.Compose([
    A.RandomResizedCrop(IMG_SIZE, IMG_SIZE, scale=(0.85, 1.1)),
    A.RandomRotate90(),
    A.Flip(),
    A.Transpose(),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2(),
])

valid_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2(),
])

In [8]:
class PetDataset(Dataset):
    def __init__(self, df, data_dir, transform=None, mode='train'):
        self.df = df
        self.data_dir = data_dir
        self.transform = transform
        self.mode = mode
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = str(self.data_dir) + "/" + str(row[ID_COL]) + ".jpg"
        img = np.array(Image.open(img_path).convert('RGB'))
        if self.transform is not None:
            img = self.transform(image=img)['image']
        tgt = row[TARGET_COL] if self.mode == 'train' else 0
        return img.float().to(device), torch.tensor(tgt).float().to(device)

In [9]:
class CustomModel(nn.Module):
    def __init__(self, model_name=MODEL_NAME, pretrained=True):
        super(CustomModel, self).__init__()
        self.model = timm.create_model(MODEL_NAME)
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.head = create_head(self.model.num_features, 1)
        self.act = nn.ReLU()
        
    def forward(self, x):
        x = self.model(x)
        x = self.head(x)
        x = self.act(x)
        return x

In [10]:
def inference(model, dataloader):
    with torch.no_grad():
        model.eval()
        scores = 0.0
        preds = []
        for x, target in dataloader:
            preds.append(model(x).view(-1).detach().cpu().numpy())
    #         scores += mean_squared_error(target.detach().cpu().numpy(), preds, squared=False)

    #     print('Validation RMSE Loss: {:.4f} '.format(scores / len(dataloader)))

        return np.array(preds).flatten()

In [11]:
def generate_submission(valid_transform, test_img_path='../input/petfinder-pawpularity-score/test/', model_path='../input/modelsfinal/model_final.pth'):
    test_df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
#     train_data = PetDataset(train_df, TRAIN_IMG_PATH, train_transform)
#     train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    
    test_data = PetDataset(test_df, test_img_path, valid_transform, mode='test')
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False)
    
    model = CustomModel(MODEL_NAME, pretrained=False).to(device)
    model.load_state_dict(torch.load(model_path))
#     final_predictions = np.zeros_like(test_df["Id"])
#     print(final_predictions)
    final_predictions = inference(model, test_loader)
    print(final_predictions)

    test_df["Pawpularity"] = final_predictions
    test_df = test_df[["Id", "Pawpularity"]]
    test_df.to_csv("submission.csv", index=False)
#     sample_submission.Pawpularity = final_predictions
#     sample_submission.to_csv("submission.csv", index=False)
    
#     learner.save(f'learner_fold_{fold}')
#     torch.save(learner.model.state_dict(), f'./train_all.pth')

In [12]:
generate_submission(valid_transform)